## Installing necessary libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 300)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pointbiserialr
import pingouin as pg
import matplotlib.pyplot as plt

/home/jupyter-apaddo/.local/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Loading the merged dataframe

In [2]:
woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")
woundassessment=woundassessment.drop(["Unnamed: 0"],axis=1)

#WoundLocation are stored in Wound_Title column
woundassessment=woundassessment.rename(columns={"Wound_Title":"WoundLocation"})
woundassessment.columns.tolist()

#Dropping some columns which are not relevant for classification
woundassessment=woundassessment.drop(["WoundID", "PatientID", "WoundNumber"],axis=1)

#PatientIDWoundNumber is showing a floating point number having a 0 added to it, changing it.
woundassessment['PatientIDWoundNumber'] = woundassessment['PatientIDWoundNumber'].astype(str).str.replace('\.0', '', regex=True)

#Dropping duplicates
woundassessment=woundassessment.drop_duplicates()

woundassessment=woundassessment.reset_index()
woundassessment=woundassessment.drop(["index"],axis=1)

/tmp/ipykernel_149724/3912794564.py:1: DtypeWarning: Columns (61,62,63,64,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,141,147,154) have mixed types. Specify dtype option on import or set low_memory=False.
  woundassessment=pd.read_csv("Final_Merged_DataFrame_June24.csv")


### Check the Wound Location

In [3]:
woundassessment["WoundLocation"]

0             Wound #1 Right, Plantar Foot
1             Wound #1 Right, Plantar Foot
2             Wound #1 Right, Plantar Foot
3             Wound #1 Right, Plantar Foot
4             Wound #1 Right, Plantar Foot
                        ...               
309061       Wound #2 Right, Plantar Foot 
309062       Wound #2 Right, Plantar Foot 
309063    Wound #3 Right, Posterior Ankle 
309064    Wound #3 Right, Posterior Ankle 
309065    Wound #3 Right, Posterior Ankle 
Name: WoundLocation, Length: 309066, dtype: object

### We don't need left-right information here, thus removing that

In [4]:
# Correcting the Wound Location values
woundassessment['WoundLocation'] = woundassessment['WoundLocation'].str.split(', ').str[1].str.strip()
woundassessment["WoundLocation"]

0            Plantar Foot
1            Plantar Foot
2            Plantar Foot
3            Plantar Foot
4            Plantar Foot
               ...       
309061       Plantar Foot
309062       Plantar Foot
309063    Posterior Ankle
309064    Posterior Ankle
309065    Posterior Ankle
Name: WoundLocation, Length: 309066, dtype: object

## Dropping the 'ID' columns because we do not need those for classification and few columns which are not relevant to predict wound healing status

In [22]:
woundassessment=woundassessment.drop(["Woundassessment_UserID","patientmedications_DocumentType","patientmedications_VisitID",
                                      "patientmedications_UserID","Wound_DocumentType","Wound_UserID","Wound_PhysicianID", 
                                      "AdmissionID", "previousID", "SyncUserID","ReferringPhysicianID", "patientmedications_Title", 
                                      "patientmedications_PhysicianID", "Woundassessment_PhysicianID","Patient_UserID","PhysicianID",
                                      "Woundassessment_VisitID", "Notes", "imageFileCount", "Woundassessment_Title", 
                                      "Woundassessment_DocumentType"], axis=1)
woundassessment

,Pulse,AdherentYellowSlough,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,...,SmokingStatusText,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,TInsuranceZip,PInsuranceZip,Diabetes
0,NaN,NaN,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,...,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,NaN,NaN,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309062,NaN,NaN,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309063,NaN,NaN,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309064,NaN,NaN,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,...,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Check which columns have how many non-null values

In [23]:
# Get the number of non-null values in each column
non_null_counts = woundassessment.count()

# Get the number of null values in each column
null_counts = woundassessment.isnull().sum()

# Output the counts
print("Non-null counts:")
#print(non_null_counts)
#print("\nNull counts:")
#print(null_counts)

non_null_counts

Non-null counts:


Pulse                                     71072
AdherentYellowSlough                       2176
AdiposeExposed                           303691
AdiposeNecrosisExposed                   303662
area                                     258097
AtrophieBlanche                          303692
BoneNecrosisExposed                      303662
BrawnyInduration                         303692
Callus                                   303692
CapsuleExposed                           303663
CapsuleNecrosisExposed                   303662
color                                    207881
CompCompression                          175054
CompDiet                                 284040
CompDressing                             284040
CompExercise                             284040
CompHBOVisits                            284039
CompLimb                                 284040
CompMedication                           284040
CompOffLoad                              284040
CompSmoking                             

## Drop those columns which have less than 5000 non-values

In [24]:
woundassessment = woundassessment.dropna(axis=1, thresh=5000)
woundassessment

,Pulse,AdiposeExposed,AdiposeNecrosisExposed,area,AtrophieBlanche,BoneNecrosisExposed,BrawnyInduration,Callus,CapsuleExposed,CapsuleNecrosisExposed,...,BMI,SmokingStatus,SmokingStatusText,PatientIDWoundNumber,DebridementLevel,Method,Bleeding,IsThisBurn,SInsuranceZip,PInsuranceZip
0,NaN,1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
1,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
2,NaN,-1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
3,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
4,NaN,-1.0,1.0,400.0,1.0,1.0,1.0,1.0,1.0,1.0,...,27.5,3.0,quit 1992,5978401_1,7.0,79.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309061,NaN,1.0,1.0,93.7,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN
309062,NaN,1.0,1.0,46.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_2,NaN,NaN,NaN,NaN,NaN,NaN
309063,NaN,1.0,1.0,81.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN
309064,NaN,1.0,1.0,67.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10210112_3,NaN,NaN,NaN,NaN,NaN,NaN


## See how many unique wounds are there for which we want to do the classification and sort the dataframe based on the WoundNumber

In [25]:
display(woundassessment["PatientIDWoundNumber"].nunique())
sorted_df = woundassessment.sort_values('PatientIDWoundNumber')

25255

## See how many unique patients are there for whom we want to do the classification

In [26]:
sorted_df["PatientIDWoundNumber"].str.split('_').str[0].nunique()

8728

#### We want to have a processed dataframe for the classification but in this dataframe there are a lot of data elements, so we want to keep only those data elements which are necessary/important for the classification models. Thus we need to do the feature importance, thus we are adding new column in this "wound healing status", using that as target variable, we will try to compute feature importance value for each data elements and based on that, we will remove the data elements before doing the transpose.

In [ ]:
big_df=pd.DataFrame()


sorted_df=sorted_df[sorted_df.DateAdded_woundassessment.notnull()]
#sorted_df["area"]=sorted_df["area"].replace(0,0.00001)

unique_wounds=sorted_df["PatientIDWoundNumber"].unique().tolist()
for i in range(len(unique_wounds)):
#     if i>500:
#         break
    print(i)
    
    wound=unique_wounds[i]
    df_wound=sorted_df.query("PatientIDWoundNumber==@wound")
    #df_wound=df_wound[["PatientIDWoundNumber","DateAdded_woundassessment","area"]]
    df_wound=df_wound.drop_duplicates()
    df_wound=df_wound[df_wound.area.notnull()]

    
    # Convert 'DateAdded_woundassessment' column to datetime type
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Keep the maximum value in 'area' for each date
    df_wound = df_wound.loc[df_wound.groupby('DateAdded_woundassessment')['area'].idxmax()]

    df_wound=df_wound.sort_values('DateAdded_woundassessment')
    
    df_wound["Wound_healing_status"]=np.nan
    df_wound["Wound_healing_percentage"]=np.nan

    
    df_wound=df_wound.reset_index()
    df_wound=df_wound.drop(["index"],axis=1)
    
    for j in range(df_wound.shape[0]):
        if j == 0:
            df_wound["Wound_healing_status"][j]='Unknown'
        else:
            percentage=(df_wound["area"][j-1]-df_wound["area"][j])/df_wound["area"][j-1]*100
            df_wound["Wound_healing_percentage"][j]= percentage

            if percentage == 100:
                df_wound["Wound_healing_status"][j]= 'Healed'
            elif percentage>=65:
                df_wound["Wound_healing_status"][j]= 'Healing'
            elif (percentage<65) & (percentage>=20):
                df_wound["Wound_healing_status"][j]= 'Grey'
            else:
                df_wound["Wound_healing_status"][j]= 'Not healing'
    
    # Convert the 'Date' column to datetime
    df_wound['DateAdded_woundassessment'] = pd.to_datetime(df_wound['DateAdded_woundassessment'])

    # Find the minimum date in the DataFrame
    first_date = df_wound['DateAdded_woundassessment'].min()

    # Calculate the day differences relative to the first date
    df_wound['DayNumber'] = (df_wound['DateAdded_woundassessment'] - first_date).dt.days + 1
    
    
    big_df = pd.concat([big_df, df_wound], axis=0)

    #display(df_wound)
#sorted_df[["PatientIDWoundNumber","DateAdded_woundassessment","area"]].head(50)

In [32]:
big_df["Wound_healing_status"].unique()

array(['Unknown', 'Not healing', 'Grey', 'Healing', 'Healed'],
      dtype=object)

## We do not need the wound healing percentage column, thus we will remove that.

In [ ]:
big_df=big_df.drop(["Wound_healing_percentage"], axis=1)

## Save the dataframe

In [30]:
big_df.to_csv("big_df.csv")